# Paillier Homomorphic Encryption Example

DISCLAIMER: This is a proof-of-concept implementation. It does not represent a remotely product ready implementation or follow proper conventions for security, convenience, or scalability. It is part of a broader proof-of-concept demonstrating the vision of the OpenMined project, its major moving parts, and how they might work together.



In [1]:
from syft import he
import numpy as np

IndexError: list index out of range

# Basic Ops

In [3]:
pubkey,prikey = he.Paillier.KeyPair().generate()

In [4]:
x = pubkey.encrypt(np.array([1.,2.,3.,4.,5.]))

In [5]:
prikey.decrypt(x)

array([ 1.,  2.,  3.,  4.,  5.])

In [6]:
prikey.decrypt(x+x[0])

array([ 2.,  3.,  4.,  5.,  6.])

In [7]:
prikey.decrypt(x*5)

array([  5.,  10.,  15.,  20.,  25.])

In [8]:
prikey.decrypt(x+x/5)

array([ 1.2,  2.4,  3.6,  4.8,  6. ])

# Key SerDe

In [10]:
pubkey,prikey = he.Paillier.KeyPair().generate()

In [11]:
x = pubkey.encrypt(np.array([1.,2.,3.,4.,5.]))

In [12]:
pubkey_str = pubkey.serialize()
prikey_str = prikey.serialize()

In [15]:
pubkey2,prikey2 = he.Paillier.KeyPair().deserialize(pubkey_str,prikey_str)

In [16]:
prikey2.decrypt(x)

array([ 1.,  2.,  3.,  4.,  5.])

In [17]:
y = pubkey2.encrypt(np.ones(5))/2

In [18]:
prikey.decrypt(y)

array([ 0.5,  0.5,  0.5,  0.5,  0.5])

# Value SerDe

In [19]:
import pickle

In [20]:
y_str = pickle.dumps(y)

In [21]:
y2 = pickle.loads(y_str)

In [22]:
prikey.decrypt(y)

array([ 0.5,  0.5,  0.5,  0.5,  0.5])